In [1]:
import csv
import os
import numpy as np
import pandas as pd
import gc
from datetime import timedelta, date
import uuid
import traceback

import config
import queries as q
import tools

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
def upd_main_with_t30():
    select_qry = """
    select distinct header_id from trafc.electronic_count_data_partitioned
    where light_motor_vehicles is null
    ;
    """

    df = pd.read_sql(select_qry, config.ENGINE)
    df.dropna(subset=["header_id"], inplace=True)
    list = df.header_id.astype(str).to_list()
    cs1 = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_1, config.ENGINE)
    cs5 = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_5, config.ENGINE)
    cs8 = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_8, config.ENGINE)
    for header_id in list:
        df30 = pd.read_sql(f"""
            select * from trafc.electronic_count_data_type_30 where header_id = '{header_id}';
            """
            , config.ENGINE)
        

        df30 = df30.groupby(['site_id', 'start_datetime', 'lane_number', 'classification_scheme', 'class_number'])['number_of_vehicles'].sum()

        for index, row in df30.iteritems():
            site_id = index[0]
            start_datetime = index[1]
            lane_number = index[2]
            classification_scheme = index[3]
            class_number = index[4]
            number_of_vehicles =  row
            if classification_scheme == 1:
                update_qry = f"""
                    update trafc.electronic_count_data_partitioned set
                    {cs1['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} = {number_of_vehicles}
                    where site_id = '{site_id}'
                    and start_datetime = '{start_datetime}'
                    and lane_number = {lane_number}
                    and {cs1['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} is null;
                """
            elif classification_scheme == 5:
                update_qry = f"""
                    update trafc.electronic_count_data_partitioned set 
                    {cs5['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} = {number_of_vehicles}
                    where site_id = '{site_id}' 
                    and start_datetime = '{start_datetime}'
                    and lane_number = {lane_number}
                    and {cs5['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} is null;            
                """
            elif classification_scheme == 8:
                update_qry = f"""
                    update trafc.electronic_count_data_partitioned set 
                    {cs8['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} = {number_of_vehicles}
                    where site_id = '{site_id}' 
                    and start_datetime = '{start_datetime}'
                    and lane_number = {lane_number}
                    and {cs8['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} is null;            
                """
            else:
                pass
            try:
                with config.CONN as conn:
                    cur = conn.cursor()
                    cur.execute(update_qry)
                    conn.commit()
            except Exception as e:
                print(e)
                print(update_qry)
                traceback.print_exc()
                conn.rollback()
                conn.close()
                gc.collect()
                break
    print("Done")

In [4]:
def get_list_of_files():
    """
    It returns a list of strings, each string being a unique header_id from a table in a database.
    :return: A list of strings.
    """
    select_qry = """
    select distinct header_id from trafc.electronic_count_data_partitioned
    where light_motor_vehicles is null
    ;
    """
    df = pd.read_sql(select_qry, config.ENGINE)
    df.dropna(subset=["header_id"], inplace=True)
    return df["header_id"].astype(str).to_list()

In [5]:
headers = get_list_of_files()

In [6]:
headers

['3079d631-5d5c-4f6e-9842-eb6c034f0fd6',
 '780b2814-1b85-4ad2-9f5c-4fa9d996da4c',
 '6ff09515-f329-48eb-af17-83a6e79ea0e0',
 'e7a4e5d3-b2df-407a-96b6-4d6acbeb9741',
 'e4f13613-a043-498d-8fff-8d5c7666eccb',
 'f10aa79d-7154-48ec-ba3c-3ee31ac3af25',
 '17e61368-f64b-49c0-a3fe-769d026bf097',
 '3fcde8de-69c1-42d8-9704-548dc5ca457d',
 '5e6e0029-ae41-433b-b009-399a933583ea',
 'c48acb3c-e19e-4d90-b82a-b4e456dbc308',
 '006f020e-6ecf-4d2e-9e43-283500027e62',
 'b6c92b36-83f7-41e3-8bec-a853b01cdc51',
 '1bbbf09a-96e3-4578-a327-bf880b0e4b27',
 '74657133-b2b0-4b8d-9a62-1a178a9dc0c7',
 '1412592c-5d74-4796-b3d2-52df081721f2',
 'f2190bf6-22f7-4efc-a0ec-a922ae763eaf',
 'aea0b6b8-516b-4033-92ee-c7ac599c865a',
 '66b25eda-8ae1-45c0-9b92-6121acb97024',
 '2d2ab33a-7f85-4120-947a-e1d9419bf61c',
 'cc6750dc-5a8d-461d-ad18-b548c36f29eb',
 '68197925-796d-4387-8d72-67abddfbd736',
 'd85a343a-69d7-46aa-abe2-61b6eae72741',
 'f2f61e60-3f30-45fd-ad11-e8baa514db06',
 '1616ef9c-a0f8-4a9a-bcf7-4e70fa6955a1',
 '30963bb2-5c08-